## Text Preprocessing ##

Tutorial ini membahas basic class untuk preprocessing text.

Misalkan kita memiliki list teks, misalnya daftar quotes top film.

In [1]:
import pandas as pd

df = pd.read_csv("movie_quotes.csv", encoding='utf-8')

print(df.head())

   #                                              QUOTE               MOVIE  \
0  1             Frankly, my dear, I don't give a damn.  GONE WITH THE WIND   
1  2       I'm gonna make him an offer he can't refuse.       THE GODFATHER   
2  3  You don't understand!  I coulda had class. I c...   ON THE WATERFRONT   
3  4  Toto, I've a feeling we're not in Kansas anymore.    THE WIZARD OF OZ   
4  5                        Here's looking at you, kid.          CASABLANCA   

   YEAR                     Director  
0  1939          Victor Fleming       
1  1972    Francis Ford Coppola       
2  1954              Elia Kazan       
3  1939          Victor Fleming       
4  1942          Michael Curtiz       


kita akan membangun class yang membersihkan/cleans quotes dan menghasilkan encoding one-hot atau matriks TF-IDF.

Biasanya, data teks dibersihkan dengan menghapus elemen yang tidak diperlukan. Misalnya, kata-kata seperti saya, aku, gw, kamu, muncul dalam banyak teks dan hampir tidak informatif. Ini disebut stopwords. Kuta harus menghapus ini. Kedua, untuk membuat corpus teks yang lebih homogen, kita bisa menurunkan huruf kecil semua karakter sehingga misal kata "Sintha sayangku" identik dengan "sintha sayangku". Terakhir, dalam tutorial ini kita akan stem kata-kata tersebut ke bentuk akarnya. Ini berarti kita akan mengubah kata-kata seperti memancing,dan kepancing menjadi akar kata pancing.

Class dalam Python mirip dengan membuat pemotong kue. Kita ingin membuat prototipe agar setiap kita menggunakan cookie cutter pada dataset teks, objek tersebut akan memiliki bentuk, karakteristik, dan fungsi yang sama.

Mari kita mulai dengan menginisialisasi class kita.

In [2]:
from nltk import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import download

class Preprocess():
	def __init__(self, text, sw=stopwords.words('english'), lower=True, stem = True):

		if not (type(text)==pd.core.series.Series):
			text = pd.Series(text)

		self.text = text
		self.sw = sw
		self.lower = lower
		self.stem = stem

Setiap class pada Python harus <i>diinisialisasi</i>. Pada dasarnya, kita ingin menggunakan fungsi <b>$__init__$</b> (yaitu dua garis bawah sebelum dan setelah kata init) untuk memberi objek beberapa karakteristik dasar. Dalam kasus kita, ini akan menjadi karakteristik bagaimana kita ingin membersihkan/cleaning data.

<b>self</b> adalah elemen penting dari program berorientasi objek. Ini adalah class itu sendiri dan akan menjadi variabel yang dapat kita tambahkan atributnya.

<b>text</b> akan menjadi daftar teks yang ingin kita bersihkan. Variabel ini wajib dimasukkan oleh pengguna saat memanggil kelas Preprocess yang kita buat.

<b>sw</b> adalah daftar stopwords. Kamu akan melihat ini sama dengan nilai (stopwords.words ('english')). Ini berarti secara default stopwords akan diimpor dari modul nltk (alat bahasa alami yang bagus yang dibuat oleh orang-orang baik di UPenn <a>http://www.nltk.org</a>). Ini berarti sw TIDAK wajib karena akan mengambil nilai secara default jika pengguna tidak memberinya nilai.

<b>lower</b> adalah variabel Boolean secara default sama dengan True. Kita akan menggunakan ini nanti untuk mengetahui apakah teks harus huruf kecil atau tidak.

<b>stem</b>  adalah variabel Boolean secara default sama dengan True. Kita akan menggunakan ini nanti untuk mengetahui apakah akan stem kata-kata dalam teks atau tidak.

Baris terakhir seperti <i>self.text = text</i> adalah atribut pegging ke objek kita. Artinya, di tempat lain di dalam kelas, kita dapat merujuk ke atribut ini dengan menjalankan/running <i>class_instance</i>.text.

(type (text) == pd.core.series.Series) memeriksa apakah teks tersebut adalah seri Pandas/pandas series. Jika bukan, maka kita akan mengconvert menjadi satu. Pandas adalah modul yang memudahkan untuk memanipulasi list, misalnya menurunkan casing atau mengembalikan dataframe.

Berikut contohnya:

In [3]:
docs = Preprocess(df.QUOTE)

print(docs.text.head())
print(docs.lower)
print(docs.sw)

0               Frankly, my dear, I don't give a damn.
1         I'm gonna make him an offer he can't refuse.
2    You don't understand!  I coulda had class. I c...
3    Toto, I've a feeling we're not in Kansas anymore.
4                          Here's looking at you, kid.
Name: QUOTE, dtype: object
True
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'a

Pertama kita membuat <i>instance</i> kelas, lalu kita memanggil atributnya (teks yang saya cetak hanya bagian atas, atau head, dari dataframe untuk menghemat ruang).

Sekarang kita memiliki fungsi $__ init __$, mari kita definisikan <b>method</b> pertama kita. Method adalah fungsi di dalam class. Argumen pertama harus self, karena variabel ini berisi atribut yang kita definisikan dalam fungsi $__ init __$. Pertama, kita akan memeriksa apakah atribut <i>lower</i> adalah True; jika demikian, mari kita kecilkan teksnya. Kemudian, mari kita pisahkan setiap teks menjadi list kata. Dengan cara ini kita dapat menggilir setiap kata dan 
i) stem jika self.stem True 
ii) Menghapusnya.remove jika itu stopword. 

Mirip dengan fungsi Python lainnya, kita dapat mendefinisikan fungsi di dalam fungsi untuk membuat kode yang lebih bersih. Terakhir, kita akan menggabungkan setiap list kata untuk membuat string, dan kemudian menginisialisasi objek TfidfVectorizer. TfidfVectorizer akan memungkinkan kita membuat matriks padat di mana setiap kolom adalah kata dalam kosakata kita, dan setiap baris sesuai dengan dokumen.

In [4]:
class Preprocess():
	def __init__(self, text, sw=stopwords.words('english'), lower=True, stem = True):

		if not (type(text)==pd.core.series.Series):
			text = pd.Series(text)

		self.text = text
		self.sw = sw
		self.lower = lower
		self.stem = stem


	def clean_text(self):
		def stem(word_list):
			return map(lambda x: PorterStemmer().stem(x), word_list)

		def remove_sw(word_list):
			keep = []
			for word in word_list:
				if not word in self.sw:
					keep.append(word)
			return keep

		if self.lower:
			self.text = self.text.str.lower()

		self.text = self.text.apply(lambda x: x.split())
		
		if self.stem: self.text = self.text.apply(stem)
		if self.sw: self.text = self.text.apply(remove_sw)

		self.text = self.text.apply(lambda x: ' '.join(x))
		self.vectorizer = TfidfVectorizer()
		self.df_dense = self.vectorizer.fit_transform(self.text)


In [5]:
docs = Preprocess(df.QUOTE)
docs.clean_text()

print(docs.text.head())
print(docs.vectorizer)
print(docs.df_dense[0:2])

0                            frankly, dear, give damn.
1                   i'm gonna make offer can't refuse.
2    understand! coulda class. coulda contender. co...
3                   toto, i'v feel we'r kansa anymore.
4                                 here' look you, kid.
Name: QUOTE, dtype: object
TfidfVectorizer()
  (0, 70)	0.478079705674071
  (0, 111)	0.5209988435904572
  (0, 73)	0.478079705674071
  (0, 104)	0.5209988435904572
  (1, 235)	0.5008150769034725
  (1, 49)	0.407581696306535
  (1, 209)	0.5008150769034725
  (1, 175)	0.407581696306535
  (1, 115)	0.407581696306535


Atribut <b>df_dense</b> adalah representasi dense dari matriks TF-IDF (<a>https://en.wikipedia.org/wiki/Tf%E2%80%93idf).</a>. (0, 70) sesuai dengan dokumen pertama, "frankly, dear, I don't give a damn.", Dan kata ke-71 dalam kosakata kita yang terkandung dalam kutipan itu. 0.478079... adalah skor TF-IDF.

Kita mungkin ingin mengembalikan ini dalam bentuk kumpulan data yang lebih mudah dibaca. Jadi mari kita buat dataframe Pandas di mana kolomnya adalah kata-kata dalam kosakata kita setelah membersihkan teks, self.vectorizer.get_feature_names(), dan kemudian nilai yang bisa kita dapatkan dari self.df_dense.toarray().

<b>onehot</b> akan menjadi variabel yang dapat kita gunakan sehingga pengguna dapat mengembalikan matriks dengan nilai 1 jika dokumen tersebut berisi kata dalam kosa kata di posisi ke-i, atau skor TF-IDF.

In [6]:
import pandas as pd
from nltk import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import download

# download('stopwords')

df = pd.read_csv("movie_quotes.csv", encoding='utf-8')

class Preprocess():
	def __init__(self, text, sw=stopwords.words('english'), lower=True, stem = True):

		if not (type(text)==pd.core.series.Series):
			text = pd.Series(text)

		self.text = text
		self.sw = sw
		self.lower = lower
		self.stem = stem


	def clean_text(self):
		def stem(word_list):
			return map(lambda x: PorterStemmer().stem(x), word_list)

		def remove_sw(word_list):
			keep = []
			for word in word_list:
				if not word in self.sw:
					keep.append(word)
			return keep

		if self.lower:
			self.text = self.text.str.lower()

		self.text = self.text.apply(lambda x: x.split())
		
		if self.stem: self.text = self.text.apply(stem)
		if self.sw: self.text = self.text.apply(remove_sw)

		self.text = self.text.apply(lambda x: ' '.join(x))
		self.vectorizer = TfidfVectorizer()
		self.df_dense = self.vectorizer.fit_transform(self.text)

	def array(self, onehot=1):
		array = self.df_dense.toarray().copy()
		if onehot:
			array[array>0] = 1
		return array

	def make_df(self,onehot=1):
		df = pd.DataFrame(columns=self.vectorizer.get_feature_names(),
							data = self.array(onehot))
		df['Text'] = self.text
		df = df[['Text']+list(df.columns[:-1])]
		return df

docs = Preprocess(df.QUOTE)
docs.clean_text()
text_df = docs.make_df(onehot=1)

print(text_df.head())

                                                Text  adrian  again  ahead  \
0                          frankly, dear, give damn.     0.0    0.0    0.0   
1                 i'm gonna make offer can't refuse.     0.0    0.0    0.0   
2  understand! coulda class. coulda contender. co...     0.0    0.0    0.0   
3                 toto, i'v feel we'r kansa anymore.     0.0    0.0    0.0   
4                               here' look you, kid.     0.0    0.0    0.0   

   ain  airplanes  alive  all  alone  alway  ...  win  wire  witness  word  \
0  0.0        0.0    0.0  0.0    0.0    0.0  ...  0.0   0.0      0.0   0.0   
1  0.0        0.0    0.0  0.0    0.0    0.0  ...  0.0   0.0      0.0   0.0   
2  0.0        0.0    0.0  0.0    0.0    0.0  ...  0.0   0.0      0.0   0.0   
3  0.0        0.0    0.0  0.0    0.0    0.0  ...  0.0   0.0      0.0   0.0   
4  0.0        0.0    0.0  0.0    0.0    0.0  ...  0.0   0.0      0.0   0.0   

   world   ya  yet   yo  you  youngster  
0    0.0  0.0  0.0  

Dari hasil diatas kita akan coba convert dalam bentuk csv file

In [7]:
text_df.to_csv (r'export_dataframe.csv', index = False, header=True)